# Исследование моделей

## Библиотеки

In [1]:
import os
import numpy as np
import pandas as pd
import datatable as dt
from typing import List, Union, Dict, Tuple

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error

import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Загрузка данных, предобработка

In [2]:
# Пути к файлам с данными.
chronom_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'chronom_train.csv')
chronom_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'chronom_test.csv')

chugun_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'chugun_train.csv')
chugun_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'chugun_test.csv')

gas_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'gas_train.csv')
gas_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'gas_test.csv')

lom_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'lom_train.csv')
lom_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'lom_test.csv')

plavki_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'plavki_train.csv')
plavki_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'plavki_test.csv')

produv_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'produv_train.csv')
produv_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'produv_test.csv')

sip_train_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'sip_train.csv')
sip_test_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'sip_test.csv')

target_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'target_train.csv')

submission_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'sample_submission.csv')

In [3]:
# Загрузка данных.
chronom_train = dt.fread(chronom_train_path).to_pandas()
chronom_train.name = 'chronom_train'
chronom_test = dt.fread(chronom_test_path).to_pandas()
chronom_test.name = 'chronom_test'

chugun_train = dt.fread(chugun_train_path).to_pandas()
chugun_train.name = 'chugun_train'
chugun_test = dt.fread(chugun_test_path).to_pandas()
chugun_test.name = 'chugun_test'

gas_train = dt.fread(gas_train_path).to_pandas()
gas_train.name = 'gas_train'
gas_test = dt.fread(gas_test_path).to_pandas()
gas_test.name = 'gas_test'

lom_train = dt.fread(lom_train_path).to_pandas()
lom_train.name = 'lom_train'
lom_test = dt.fread(lom_test_path).to_pandas()
lom_test.name = 'lom_test'

plavki_train = dt.fread(plavki_train_path).to_pandas()
plavki_train.name = 'plavki_train'
plavki_test = dt.fread(plavki_test_path).to_pandas()
plavki_test.name = 'plavki_test'

produv_train = dt.fread(produv_train_path).to_pandas()
produv_train.name = 'produv_train'
produv_test = dt.fread(produv_test_path).to_pandas()
produv_test.name = 'produv_test'

sip_train = dt.fread(sip_train_path).to_pandas()
sip_train.name = 'sip_train'
sip_test = dt.fread(sip_test_path).to_pandas()
sip_test.name = 'v'

target = dt.fread(target_path).to_pandas()
target.name = 'target'

submission = dt.fread(submission_path).to_pandas()
submission.name = 'submission'

In [4]:
# Списки загруженных файлов.
all_train = [chronom_train, chugun_train, gas_train, lom_train, plavki_train, produv_train, sip_train, target]
all_test = [chronom_test, chugun_test, gas_test, lom_test, plavki_test, produv_test, sip_test]

### Обучающая выборка

In [5]:
def check_shapes_and_missings(dfs: List[pd.DataFrame]) -> None:
    # Статистика по всем загруженным файлам.
    cols = ['Таблица', 'Объекты', 'Признаки', 'Пропуски (всего)', 'Признак с аибольшим числом пропусков']
    stats = pd.DataFrame(columns=cols)
    
    for df in dfs:
        stats = stats.append(
            pd.DataFrame(
                data=[[df.name, df.shape[0], df.shape[1], 
                       df.isna().sum().sum(), (df.count().idxmin(), df[df.count().idxmin()].isna().sum())]],
                columns=cols)
        )
    
    return stats.reset_index(drop=True) 

In [6]:
check_shapes_and_missings(all_train)

,Таблица,Объекты,Признаки,Пропуски (всего),Признак с аибольшим числом пропусков
0,chronom_train,34406,7,31490,"(O2, 31490)"
1,chugun_train,2063,13,0,"(NPLV, 0)"
2,gas_train,6468018,13,0,"(NPLV, 0)"
3,lom_train,6376,4,0,"(NPLV, 0)"
4,plavki_train,2137,10,0,"(NPLV, 0)"
5,produv_train,4729802,4,0,"(NPLV, 0)"
6,sip_train,31584,5,0,"(NPLV, 0)"
7,target,2063,3,2,"(C, 2)"


In [9]:
# Преобразовать время до секунд.
produv_train['SEC'] = pd.to_datetime(produv_train['SEC']).astype('datetime64[s]')
gas_train['Time'] = pd.to_datetime(gas_train['Time']).astype('datetime64[s]')

In [10]:
# Соединить gas и produv по inner join.
produv_gas_combined = pd.merge(produv_train, gas_train, how='inner', 
                               left_on=['NPLV', 'SEC'], right_on=['NPLV', 'Time'])

In [11]:
# Отобрать процесс продувки для каждой группы из cronom.
chronom_groups = chronom_train.groupby(["NPLV"])
cronom_produv = chronom_groups.apply(lambda g: g[g['NOP'] == 'Продувка']).reset_index(drop=True)
# Отобрать время окночания операции по группам.
cronom_produv = cronom_produv[['NPLV', 'VR_KON']]
cronom_produv['VR_KON'] = cronom_produv['VR_KON'].astype('datetime64[s]')

In [12]:
# Соединить produv_gas_combined и cronom_produv по inner join.
prod_gas_cron = pd.merge(produv_gas_combined, cronom_produv, how='inner', 
                         left_on=['NPLV'], right_on=['NPLV'])

In [13]:
# Обрезать данные по времени завершения продувки.
groups_pgc = prod_gas_cron.groupby(['NPLV'])
pgc_cut = groups_pgc.apply(lambda g: g[g['SEC'] <= g['VR_KON']]).reset_index(drop=True)

In [17]:
# Отобрать последние 500 записей по времени перед окончанием процесса продувки.
pgc_selected = pgc_cut.groupby(['NPLV']).tail(500)

In [25]:
# Отбросить плавки с количеством записей меенше 500.
pgc_selected = pgc_selected.groupby('NPLV').filter(lambda x: x.shape[0] == 500)

In [27]:
# Удалить лишнии признаки.
pgc_selected = pgc_selected.drop(columns=['SEC', 'VR_KON']).reset_index(drop=True)

In [49]:
# Сортировка.
pgc = pgc_selected.sort_values(by=['Time', 'NPLV'])

In [50]:
# Перевести время в длительность процесса.
pgc['min_group_time'] = pgc.groupby(['NPLV'])['Time'].transform('min')
pgc['duration'] = (pgc.Time - pgc.min_group_time).dt.components['seconds']
pgc = pgc.drop(columns=['min_group_time', 'Time'])

In [84]:
# Перевести время в длительность процесса.
pgc = pgc[~pgc["NPLV"].isin(target[target['C'].isna()]["NPLV"].values)]

,NPLV,RAS,POL,V,T,O2,N2,H2,CO2,CO,AR,T фурмы 1,T фурмы 2,O2_pressure,duration
0,510008,906.666667,1.680000,214087.218750,802.430542,0.573333,43.380001,2.14,25.980000,27.809999,0.67,0.000000,0.000000,13.382523,0
1,510008,906.833333,1.657500,213762.593750,805.902771,0.630667,43.750000,2.07,26.080000,27.430000,0.68,0.000000,0.000000,13.382523,2
2,510008,907.000000,1.635000,213111.859375,806.944458,0.688000,43.750000,2.07,26.080000,27.430000,0.68,0.000000,0.000000,13.389757,4
3,510008,907.166667,1.612500,213111.859375,809.027771,0.745333,44.220001,2.02,26.320000,26.760000,0.68,0.000000,0.000000,13.389757,6
4,510008,907.333333,1.590000,213979.046875,810.763916,0.802667,44.130001,1.98,26.320000,26.900000,0.67,0.000000,0.000000,13.375290,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023495,512322,138.000000,6.530000,207944.078125,850.347229,0.000000,55.459999,0.97,27.750000,15.060000,0.76,38.975697,26.620371,14.793113,42
1023496,512322,112.000000,6.540000,209938.265625,851.388855,0.000000,56.540001,0.88,28.059999,13.760000,0.74,38.888889,26.620371,14.800347,44
1023497,512322,92.000000,6.620000,209054.296875,852.777771,0.000000,56.669998,0.83,28.270000,13.470000,0.76,38.831020,26.620371,14.822048,46
1023498,512322,75.000000,8.320000,210708.687500,853.819458,0.000000,56.860001,0.77,28.219999,13.410000,0.75,38.773148,26.620371,14.822048,48


In [72]:
%%time
# Привеести к виду, необходимому для LSTM-модели.
temp = pgc.copy()
temp["count"] = temp.groupby("NPLV")["duration"].cumcount()
mux = pd.MultiIndex.from_product(
    [temp["NPLV"].unique(), range(max(temp["count"]+1))], names=["NPLV","count"])
temp = temp.set_index(["NPLV","count"]).reindex(mux, fill_value=0)
numpy_array = np.array(temp.groupby(level=0).apply(pd.Series.tolist).values.tolist())

CPU times: user 4.12 s, sys: 367 ms, total: 4.48 s
Wall time: 4.53 s


In [73]:
# Проверка.
numpy_array.shape

(2047, 500, 14)

In [ ]:
# Reshape all datasets to fit into models.
X_train_flat = X_train.reshape((X_train.shape[0], -1), order='F')
X_test_flat = X_test.reshape((X_test.shape[0], -1), order='F')
y_train_flat = y_temp.values
# Convert numpy array of targets into 1D categorival Pandas DataFrame.
y_temp_test = pd.DataFrame(data=[int(np.where(x==1)[0]) for x in y_test])
y_test_flat = y_temp_test.values

# Fit the model.
cb.fit(X_train_flat, y_train_flat.ravel(), plot=True, eval_set=(X_test_flat, y_test_flat))

## EDA и очистка данных

### Типы данных и пропуски

In [32]:
# Типы данных.
# data.dtypes

In [17]:
# Преобразование даты.
data.date = pd.to_datetime(data.date)

In [27]:
# Время, прошедшее с начала процесса плавки (для каждой плавки).
data['pricess_time'] = data['date'] - data.groupby('44')['date'].transform('min')
# Разница между мимнимальнй датой в датасете и датой в тякущей ячейке.
data['period_from_start'] = (data.date - data.date.min()).dt.components['days'] # .components['hours/minutes/seconds']
# Ординальная дата.
data['ordinal_date'] = data.date.apply(lambda x: x.toordinal())
# Элементы даты.
data['year'] = data.date.dt.year
data['month'] = data.date.dt.month
data['week'] = data.date.dt.week
data['year_day'] = data.date.dt.dayofyear
data['month_day'] = data.date.dt.day
data['week_day'] = data.date.dt.dayofweek
# Время (с учетом 24-часов)


## Модели

### Классические

In [ ]:
# Линейная регрессия с L2-регуляризацией.
from sklearn.linear_model import Ridge
# Случайный лес.
from sklearn.ensemble import RandomForestRegressor
# Бустниги.
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# Мультитаргет-регрессия.
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
# Пример мультитаргет-регрессия.
ridge = Ridge()
model = MultiOutputRegressor(estimator=ridge)
model.fit(X_train, y_train)

score = model.score(X_train, y_train)
print("Training score:", score)

y_pred = model.predict(X_test)

## Мета-алгоритмы

In [ ]:
# Голосование.
from sklearn.ensemble import VotingRegressor
# Пример.
voting_algorithm = VotingRegressor(
    [('CB', CatBoostRegressor()), ('XGB', XGBRegressor())]
)

y_pred = voting_algorithm.fit(X_train, y_train).predict(X_test)

In [ ]:
# Бэггинг.
from sklearn.ensemble import BaggingRegressor
# Пример.
bagging = BaggingRegressor(base_estimator=Ridge(), n_estimators=10, random_state=0).fit(X, y)

y_pred = bagging.predict(X_test)

In [ ]:
# Стэкинг.
from sklearn.ensemble import StackingRegressor
# Пример.
estimators = [
    ('CB', CatBoostRegressor()),
    ('XGB', XGBRegressor())
]
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10, random_state=42)
)

reg.fit(X_train, y_train).score(X_test, y_test)

### Нейросеть

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

## Метрики

In [ ]:
def evraz_metric(answers: pd.DataFrame, user_csv: pd.DataFrame):
    """
    Метрика оценки качества модели, предложенная организаторами EVRAZ.
    :param answers: pd.DataFrame, датасет с реальными значениями целевых переменных.
    :param user_csv: pd.DataFrame, датасет с предсказанными значениями целевых переменных.
    :return:
    """
    # Содержание углерода в металле.
    delta_c = np.abs(np.array(answers['C']) - np.array(user_csv['C']))
    hit_rate_c = np.int64(delta_c < 0.02)
    # Температура металла.
    delta_t = np.abs(np.array(answers['TST']) - np.array(user_csv['TST']))
    hit_rate_t = np.int64(delta_t < 20)

    N = np.size(answers['C'])

    return np.sum(hit_rate_c + hit_rate_t) / 2 / N


def median_absolute_percentage_error(y_true: np.array, y_pred: np.array) -> float:
    return np.median(np.abs(y_pred-y_true)/y_true)


def metrics_stat(y_true: np.array, y_pred: np.array) -> Dict[str, float]:
    """
    Вывод основных метрик.
    :param y_true: np.array, реальные значения целевой переменной.
    :param y_pred: np.array, предсказанные значения целевой переменной.
    :return: dict, словарь с названиями метрик и значениями
    """
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mdape = median_absolute_percentage_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return {'mape': mape, 'mdape': mdape, 'rmse': rmse, 'r2': r2}

## Пайплайн (классические модели)

In [ ]:
scorer = make_scorer(mape, greater_is_better=False)

lb = LabelEncoder().fit(x_train['building_type'].values)

x_train['building_type'] = lb.transform(x_train['building_type'].values)
x_test['building_type'] = lb.transform(x_test['building_type'].values)


def hyperopt(estimator, params):
    column_transformer = ColumnTransformer(  # OHE for cat, Scaler for real
        transformers=[
            ('real', StandardScaler(), real_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)

        ], n_jobs=4
    )

    pipeline = Pipeline(  # column transformer and then model
        steps=[
            ('column_transformer', column_transformer),
            ('model', estimator)

        ]
    )

    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=params,
        scoring=scorer,
        cv=3,
        verbose=2
    )

    grid.fit(x_train, y_train)

    # write best params to `best_params`
    best_params = grid.best_params_


    column_transformer = ColumnTransformer(  # OHE for cat, Scaler for real
        transformers=[
            ('real', StandardScaler(), real_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)

        ], n_jobs=4
    )

    pipeline = Pipeline(  # column transformer and then model with `best_params` as params
        steps=[
            ('column_transformer', column_transformer),
            ('model', estimator)

        ]
    )

    pipeline.set_params(**best_params)
    pipeline.fit(x_train, y_train)

    score_train = mape(y_train, pipeline.predict(x_train))
    score_test = mape(y_test, pipeline.predict(x_test))

    return score_train, score_test, best_params

In [ ]:
param_grid = {
    'model__n_estimators': [10],
    'model__learning_rate': [0.01, 0.1, 0.3, 0.5],
    'model__min_samples_split': [2, 12, 500],
    'model__max_depth': [5, 9, 12],
    'model__subsample' : [0.5, 0.8, 1],
}

hyperopt(GradientBoostingRegressor(), param_grid)

## Sample 2

In [ ]:
# Трансформеры данных
numeric_transformer = Pipeline(steps=[
    # ('imputer', SimpleImputer(strategy='median')), - можно было бы применить импьютер для заполнения пропусков
    ('scaler', StandardScaler())])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Предобработка данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)])

# Основной pipeline
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))])

# Отобразить pipeline
set_config(display='diagram')
pipe

In [ ]:
# Параметры модели и преодобработки данных
parameters = {
    'preprocessor__num__scaler': [StandardScaler(), RobustScaler()],
    'classifier__C': [100, 10, 1, 0.1, 0.01, 0.001]
}

In [ ]:
# Подбор параметров
X_train, y_train = data[num_cols + cat_cols], data[target_col]
cross_val = StratifiedShuffleSplit(n_splits=5,test_size=0.3,random_state=42)

grid = GridSearchCV(pipe, parameters, cv=cross_val, scoring='roc_auc').fit(X_train, y_train)

In [ ]:
print("Лучшие параметры:", grid.best_params_)
print("Лучший score:", grid.best_score_)

In [ ]:
# Модель
model = CatBoostClassifier(iterations=100, # количество деревьев уменьшено до 100 для ускорения расчетов
                           cat_features=cat_cols, # категориальные фичи
                           eval_metric='AUC:hints=skip_train~false', # метрика
                           verbose=False) # вывод инфорамации

# Сетка для подбора параметров
grid = {'learning_rate': [0.01, 0.03, 0.05, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

# Поиск лучших параметров
grid_search_result = model.grid_search(grid,
                                       X=X_train,
                                       y=y_train,
                                       cv=5,
                                       refit=True,
                                       shuffle=True,
                                       stratified=True,
                                       verbose=False,
                                       plot=True)

In [ ]:
# CatBoost с лучшими параметрами
best_boost = CatBoostClassifier(iterations=100,
                                cat_features=cat_cols,
                                custom_metric='AUC:hints=skip_train~false',
                                verbose=False,
                                **grid_search_result['params'])

best_boost.fit(X_train, y_train)

In [ ]:
# Результаты кросс-валидации
pd.DataFrame(grid_search_result['cv_results'])

In [ ]:
print("Лучшие параметры:", grid_search_result['params'])
print("Лучший score:", 0.8740685544)

## Пайплайн (нейросеть)

In [ ]:
def create_data(n):
    x1 = np.array([i/100+np.random.uniform(-1,3) for i in range(n)]).reshape(n,1)
    x2 = np.array([i/100+np.random.uniform(-3,5)+2 for i in range(n)]).reshape(n,1)
    x3 = np.array([i/100+np.random.uniform(-6,5)-3 for i in range(n)]).reshape(n,1)

    y1= [x1[i]-x2[i]+x3[i]+np.random.uniform(-2,2) for i in range(n)]
    y2= [x1[i]+x2[i]-x3[i]+5+np.random.uniform(-1,3) for i in range(n)]
    X = np.hstack((x1, x2, x3))
    Y = np.hstack((y1, y2))
    return X, Y

X, Y = create_data(n=450)

plt.plot(Y)
plt.show()

print("X:", X.shape, "Y:", Y.shape)
in_dim = X.shape[1]
out_dim = Y.shape[1]

xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.15)
print("xtrain:", xtrain.shape, "ytrian:", ytrain.shape)

model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", optimizer="adam")
model.summary()
 
model.fit(xtrain, ytrain, epochs=100, batch_size=12, verbose=0)
 
ypred = model.predict(xtest)
print("y1 MSE:%.4f" % mean_squared_error(ytest[:,0], ypred[:,0]))
print("y2 MSE:%.4f" % mean_squared_error(ytest[:,1], ypred[:,1]))

x_ax = range(len(xtest))
plt.scatter(x_ax, ytest[:,0],  s=6, label="y1-test")
plt.plot(x_ax, ypred[:,0], label="y1-pred")
plt.scatter(x_ax, ytest[:,1],  s=6, label="y2-test")
plt.plot(x_ax, ypred[:,1], label="y2-pred")
plt.legend()
plt.show()

In [ ]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
 
# get the dataset
def get_dataset():
	X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, n_targets=3, random_state=2)
	return X, y
 
# get the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X[train_ix], X[test_ix]
		y_train, y_test = y[train_ix], y[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results
 
# load dataset
X, y = get_dataset()
# evaluate model
results = evaluate_model(X, y)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))
